In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import pathlib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import feature_column
import tensorflow.keras.layers as layers

In [2]:
tf.__version__

'2.1.0'

In [3]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [4]:
import gc
gc.collect()

66

# 加载数据

In [5]:
data_root = pathlib.Path('/mnt/data_set/criteo_ctr')
data = pd.read_csv(data_root/'sample.csv', delimiter='\t', header=None, nrows=1e5)

In [6]:
data.shape

(100000, 40)

In [7]:
label_col = ['label']
dense_cols = ['I' + str(i) for i in range(1, 14)]
sparse_cols = ['C' + str(i) for i in range(1, 27)]
name_list = label_col + dense_cols + sparse_cols
print('dense_feats: :%s' % dense_cols)
print('sparse_feats: %s'% sparse_cols)
data.columns = name_list
data.head()

dense_feats: :['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']
sparse_feats: ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


## 预处理

In [8]:
def process_dense_feats(data, feats):
    for f in feats:
        data[f] = data[f].apply(lambda x: np.log(x+1) if x > -1 else -1)
    return data

In [9]:
data = process_dense_feats(data, dense_cols)

In [10]:
data[dense_cols] = data[dense_cols].astype(np.float32)
data[sparse_cols] = data[sparse_cols].fillna('').astype(str)

In [11]:
data.dtypes

label      int64
I1       float32
I2       float32
I3       float32
I4       float32
I5       float32
I6       float32
I7       float32
I8       float32
I9       float32
I10      float32
I11      float32
I12      float32
I13      float32
C1        object
C2        object
C3        object
C4        object
C5        object
C6        object
C7        object
C8        object
C9        object
C10       object
C11       object
C12       object
C13       object
C14       object
C15       object
C16       object
C17       object
C18       object
C19       object
C20       object
C21       object
C22       object
C23       object
C24       object
C25       object
C26       object
dtype: object

In [12]:
data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.693147,0.693147,1.791759,0.000000,7.232010,1.609438,2.772589,1.098612,5.204007,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,1.098612,0.000000,3.806663,0.693147,4.634729,2.197225,1.098612,1.098612,1.609438,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,1.098612,0.000000,0.693147,2.708050,6.643790,4.499810,1.609438,1.098612,5.505332,...,8efede7f,3412118d,,,e587c466,ad3062eb,3a171ecb,3b183c5c,,
3,0,-1.000000,6.795706,-1.000000,-1.000000,8.387768,-1.000000,0.000000,0.000000,0.000000,...,1e88c74f,74ef3502,,,6b3a5ca6,,3a171ecb,9117a34a,,
4,0,1.386294,-1.000000,-1.000000,0.000000,1.098612,0.000000,1.386294,0.000000,0.000000,...,1e88c74f,26b3c7a7,,,21c9516a,,32c7478e,b34f3128,,


In [13]:
data.nunique()

label        2
I1         153
I2        2692
I3         944
I4         136
I5       23042
I6        2056
I7         629
I8         156
I9        1943
I10          8
I11         87
I12         72
I13        274
C1         541
C2         497
C3       43870
C4       25184
C5         145
C6          12
C7        7623
C8         257
C9           3
C10      10997
C11       3799
C12      41312
C13       2796
C14         26
C15       5238
C16      34617
C17         10
C18       2548
C19       1303
C20          4
C21      38618
C22         11
C23         14
C24      12335
C25         51
C26       9527
dtype: int64

In [14]:
train, val = train_test_split(data, test_size=0.3, shuffle=False)
train.shape, val.shape

((70000, 40), (30000, 40))

In [15]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

## <----- only for test

In [16]:
batch_size = 2
train_ds = df_to_dataset(train, shuffle=False, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

In [17]:
for ds in train_ds.take(1):
    example_batch, _ = ds
example_batch

{'I1': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.6931472, 1.0986123], dtype=float32)>,
 'I2': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.6931472, 0.       ], dtype=float32)>,
 'I3': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7917595, 3.8066626], dtype=float32)>,
 'I4': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.       , 0.6931472], dtype=float32)>,
 'I5': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([7.2320104, 4.634729 ], dtype=float32)>,
 'I6': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.609438 , 2.1972246], dtype=float32)>,
 'I7': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.7725887, 1.0986123], dtype=float32)>,
 'I8': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.0986123, 1.0986123], dtype=float32)>,
 'I9': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([5.2040067, 1.609438 ], dtype=float32)>,
 'I10': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.6931472, 0.6931472], dtype=float32)>,
 'I11': <

In [18]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [19]:
I1 = feature_column.numeric_column('I1')
I1_bucket = feature_column.bucketized_column(I1, boundaries=[0,1,2])
demo(I1_bucket)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [20]:
C1 = feature_column.categorical_column_with_hash_bucket("C1", hash_bucket_size=3)
C1_onehot = feature_column.indicator_column(C1)
demo(C1_onehot)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[1. 0. 0.]
 [1. 0. 0.]]


In [21]:
C2 = feature_column.categorical_column_with_vocabulary_list("C2", vocabulary_list=data['C2'].unique())
C2_onehot = feature_column.indicator_column(C2)
demo(C2_onehot)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [22]:
cross = feature_column.crossed_column([I1_bucket, C2], hash_bucket_size=10)
demo(feature_column.indicator_column(cross))

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


## test end -------->

In [23]:
wide_feature_cols = []
deep_feature_cols = []

In [24]:
# numeric cols 
for col in dense_cols:
    wide_feature_cols.append(feature_column.numeric_column(col))
    deep_feature_cols.append(feature_column.numeric_column(col))

In [25]:
# bucketized columns
for col in dense_cols:
    num_col = feature_column.numeric_column(col)
    buck_num_col = feature_column.bucketized_column(num_col, boundaries=[1,10,100])
    wide_feature_cols.append(buck_num_col)
    deep_feature_cols.append(buck_num_col)

In [26]:
# category columns
for col in sparse_cols:
    n_uniq = data[col].nunique()
    cate_col = feature_column.categorical_column_with_vocabulary_list(col, vocabulary_list=data[col].unique())
    if n_uniq < 100:
        ## one-hot
        wide_feature_cols.append(feature_column.indicator_column(cate_col))
        deep_feature_cols.append(feature_column.indicator_column(cate_col))
    else:
        ## embedding
        cate_col = feature_column.embedding_column(cate_col, dimension=8)
        wide_feature_cols.append(cate_col)
        deep_feature_cols.append(cate_col)

In [27]:
# cross columns
for cate1, cate2 in zip(sparse_cols[:-1], sparse_cols[1:]):
    cross_col = feature_column.crossed_column([cate1, cate2], hash_bucket_size=100)
    cross_col = feature_column.indicator_column(cross_col)
    wide_feature_cols.append(cross_col)

In [28]:
wide_feature_cols[43].name

'C18_embedding'

In [29]:
for i, col in enumerate(wide_feature_cols):
    print(i, col.name, col.__class__.__name__)

0 I1 NumericColumn
1 I2 NumericColumn
2 I3 NumericColumn
3 I4 NumericColumn
4 I5 NumericColumn
5 I6 NumericColumn
6 I7 NumericColumn
7 I8 NumericColumn
8 I9 NumericColumn
9 I10 NumericColumn
10 I11 NumericColumn
11 I12 NumericColumn
12 I13 NumericColumn
13 I1_bucketized BucketizedColumn
14 I2_bucketized BucketizedColumn
15 I3_bucketized BucketizedColumn
16 I4_bucketized BucketizedColumn
17 I5_bucketized BucketizedColumn
18 I6_bucketized BucketizedColumn
19 I7_bucketized BucketizedColumn
20 I8_bucketized BucketizedColumn
21 I9_bucketized BucketizedColumn
22 I10_bucketized BucketizedColumn
23 I11_bucketized BucketizedColumn
24 I12_bucketized BucketizedColumn
25 I13_bucketized BucketizedColumn
26 C1_embedding EmbeddingColumn
27 C2_embedding EmbeddingColumn
28 C3_embedding EmbeddingColumn
29 C4_embedding EmbeddingColumn
30 C5_embedding EmbeddingColumn
31 C6_indicator IndicatorColumn
32 C7_embedding EmbeddingColumn
33 C8_embedding EmbeddingColumn
34 C9_indicator IndicatorColumn
35 C10_embed

In [30]:
wide_feature_layer = layers.DenseFeatures(wide_feature_cols)
deep_feature_layer = layers.DenseFeatures(deep_feature_cols)

In [31]:
class WideAndDeep(tf.keras.Model):
    def __init__(self, wide_feature_cols, deep_feature_cols):
        super(WideAndDeep, self).__init__()
        self.linear = layers.Dense(1, name='linear')
        self.wide_feature_layer = layers.DenseFeatures(wide_feature_cols)
        self.deep_feature_layer = layers.DenseFeatures(deep_feature_cols)
        self.deep_layers = []
        for size in [16,8,1]:
            self.deep_layers.append(layers.Dense(size, activation='relu'))
        
        self.combine_layer = layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        linear_input = self.wide_feature_layer(inputs)
        linear_out = self.linear(linear_input)
        
        deep_input = self.deep_feature_layer(inputs)
        for l in self.deep_layers:
            deep_input = l(deep_input)
        
        combine = tf.concat([linear_out, deep_input], axis=1)
        output = self.combine_layer(combine)
        return output

In [32]:
batch_size = 32
train_ds = df_to_dataset(train, shuffle=False, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

## Only FTRL

In [33]:
model = WideAndDeep(wide_feature_cols, deep_feature_cols)
auc = tf.keras.metrics.AUC()
model.compile(optimizer='ftrl', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[auc, 'accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Train for 2188 steps, validate for 938 steps
Epoch 1/5
2188/2188 [==============================] - 29s 13ms/step - loss: 0.8518 - auc: 0.5092 - accuracy: 0.7752 - val_loss: 0.8343 - val_auc: 0.5956 - val_accuracy: 0.7687
Epoch 2/5
2188/2188 [==============================] - 14s 6ms/step - loss: 0.8158 - auc: 0.5406 - accuracy: 0.7754 - val_loss: 0.7924 - val_auc: 0.5901 - val_accuracy: 0.7687
Epoch 3/5
2188/2188 [==============================] - 18s 8ms/step - loss: 0.7774 - auc: 0.5658 - accuracy: 0.7754 - val_loss: 0.7607 - val_auc: 0.5896 - val_accuracy: 0.7687
Epoch 4/5
2188/2188 [==============================] - 19s 9ms/step - loss: 0.7515 - auc: 0.5764 - accuracy: 0.7754 - val_loss: 0.7410 - val_auc: 0.5905 - val_accuracy: 0.7687
Epoch 5/5
2188/2188 [==============================] - 14s 6ms/step - loss: 0.7357 - auc: 0.5810 - accuracy: 0.7754 - val_loss: 0.7289 - val_auc: 0.5916 - val_accuracy: 0.7687


## Only Adam

In [34]:
model = WideAndDeep(wide_feature_cols, deep_feature_cols)
auc = tf.keras.metrics.AUC()
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[auc, 'accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Train for 2188 steps, validate for 938 steps
Epoch 1/5
2188/2188 [==============================] - 54s 25ms/step - loss: 0.6956 - auc_1: 0.5160 - accuracy: 0.7747 - val_loss: 0.6932 - val_auc_1: 0.5007 - val_accuracy: 0.7687
Epoch 2/5
2188/2188 [==============================] - 46s 21ms/step - loss: 0.6932 - auc_1: 0.5005 - accuracy: 0.7754 - val_loss: 0.6932 - val_auc_1: 0.5001 - val_accuracy: 0.7687
Epoch 3/5
2188/2188 [==============================] - 38s 17ms/step - loss: 0.6920 - auc_1: 0.5507 - accuracy: 0.7794 - val_loss: 0.6902 - val_auc_1: 0.6071 - val_accuracy: 0.7785
Epoch 4/5
2188/2188 [==============================] - 37s 17ms/step - loss: 0.6860 - auc_1: 0.6449 - accuracy: 0.7984 - val_loss: 0.6891 - val_auc_1: 0.6359 - val_accuracy: 0.7861
Epoch 5/5
2188/2188 [==============================] - 41s 19ms/step - loss: 0.6774 - auc_1: 0.6978 - accuracy: 0.8233 - val_loss: 0.6914 - val_auc_1: 0.6434 - val_accuracy: 0.7868


## wide部分使用ftrl，deep部分使用adam


In [35]:
loss = tf.keras.losses.BinaryCrossentropy(name='bce')
adam = tf.keras.optimizers.Adam()  ## use name or class
ftrl = tf.keras.optimizers.Ftrl()

In [36]:
train_auc = tf.keras.metrics.AUC(name='train_auc')
test_auc = tf.keras.metrics.AUC(name='test_auc')

train_bce = tf.keras.metrics.BinaryCrossentropy(name='train_bce')
test_bce = tf.keras.metrics.BinaryCrossentropy(name='test_bce')

In [37]:
model = WideAndDeep(wide_feature_cols, deep_feature_cols)

In [38]:
for x, y in train_ds.take(1):
    pred = model(x, training=False)
    print('pred ', pred)

pred  tf.Tensor(
[[0.54756373]
 [0.5382122 ]
 [0.58039093]
 [0.47321528]
 [0.561681  ]
 [0.4881842 ]
 [0.5139341 ]
 [0.50274473]
 [0.492658  ]
 [0.50318015]
 [0.52520555]
 [0.4806519 ]
 [0.49183303]
 [0.4899053 ]
 [0.5413205 ]
 [0.49208173]
 [0.48630807]
 [0.47240713]
 [0.51441056]
 [0.47781396]
 [0.48027533]
 [0.47859192]
 [0.4912872 ]
 [0.5048053 ]
 [0.49765465]
 [0.49588895]
 [0.54159915]
 [0.4905326 ]
 [0.5019788 ]
 [0.6875351 ]
 [0.4740369 ]
 [0.5002018 ]], shape=(32, 1), dtype=float32)


In [39]:
## model should build first
for weight in model.trainable_variables:
    print(weight.name)

wide_and_deep_2/linear/kernel:0
wide_and_deep_2/linear/bias:0
wide_and_deep_2/dense_features_10/C10_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C11_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C12_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C13_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C15_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C16_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C18_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C19_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C1_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C21_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C24_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C26_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C2_embedding/embedding_weights:0
wide_and_deep_2/dense_features_10/C3_embedding/embedding

In [40]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        pred = model(x, training=True)
        l = loss(y, pred)
    grads = tape.gradient(l, model.trainable_variables)
    
    linear_grads, linear_variables = [], []
    dnn_grads, dnn_variables = [], []

    for grad, variable in zip(grads, model.trainable_variables):
        if 'linear' in variable.name:
            linear_grads.append(grad)
            linear_variables.append(variable)
        else:
            dnn_grads.append(grad)
            dnn_variables.append(variable)
    
    ftrl.apply_gradients(zip(linear_grads, linear_variables))
    adam.apply_gradients(zip(dnn_grads, dnn_variables))
    
    pred = tf.squeeze(pred, axis=1)
    train_auc(y, pred)
    train_bce(y, pred)

    
@tf.function
def test_step(x, y):
    pred = model(x, training=False)
    t_l = loss(y, pred)
    pred = tf.squeeze(pred, axis=1)
    test_auc(y, pred)
    test_bce(y, pred)

In [41]:
EPOCHS = 5
template = 'Epoch {}, Loss: {}, AUC: {}, Test Loss: {}, Test AUC: {}'

for epoch in range(EPOCHS):
    train_auc.reset_states()
    test_auc.reset_states()
    
    train_bce.reset_states()
    test_bce.reset_states()
    
    for x, y in train_ds:
        train_step(x, y)
    
    for x, y in val_ds:
        test_step(x, y)

    print(template.format(epoch+1, train_bce.result(), train_auc.result(), test_bce.result(), test_auc.result()))

Epoch 1, Loss: 0.488223671913147, AUC: 0.6997932195663452, Test Loss: 0.45875999331474304, Test AUC: 0.7589820623397827
Epoch 2, Loss: 0.4196425676345825, AUC: 0.8024817705154419, Test Loss: 0.46547290682792664, Test AUC: 0.7591999769210815
Epoch 3, Loss: 0.3509247303009033, AUC: 0.8684833645820618, Test Loss: 0.5258888006210327, Test AUC: 0.7386904358863831
Epoch 4, Loss: 0.28236693143844604, AUC: 0.9146637916564941, Test Loss: 0.5852320194244385, Test AUC: 0.7269407510757446
Epoch 5, Loss: 0.23238743841648102, AUC: 0.9409314393997192, Test Loss: 0.6542839407920837, Test AUC: 0.7177342176437378


In [42]:
model.summary()

Model: "wide_and_deep_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear (Dense)               multiple                  2841      
_________________________________________________________________
dense_features_10 (DenseFeat multiple                  1929656   
_________________________________________________________________
dense_features_11 (DenseFeat multiple                  1929656   
_________________________________________________________________
dense_8 (Dense)              multiple                  5456      
_________________________________________________________________
dense_9 (Dense)              multiple                  136       
_________________________________________________________________
dense_10 (Dense)             multiple                  9         
_________________________________________________________________
dense_11 (Dense)             multiple              

In [43]:
!tensorboard --logdir ./logs

/bin/sh: 1: tensorboard: not found


## Functional API --未验证

In [44]:
# real = {
#     colname : tf.feature_column.numeric_column(colname) 
#           for colname in 
#             ('dep_delay,taxiout,distance,avg_dep_delay,avg_arr_delay' +
#              ',dep_lat,dep_lon,arr_lat,arr_lon').split(',')
# }
# sparse = {
#       'carrier': tf.feature_column.categorical_column_with_vocabulary_list('carrier',
#                   vocabulary_list='AS,VX,F9,UA,US,WN,HA,EV,MQ,DL,OO,B6,NK,AA'.split(',')),
#       'origin' : tf.feature_column.categorical_column_with_hash_bucket('origin', hash_bucket_size=1000),
#       'dest'   : tf.feature_column.categorical_column_with_hash_bucket('dest', hash_bucket_size=1000)
# }

# inputs = {
#     colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') 
#           for colname in real.keys()
# }
# inputs.update({
#     colname : tf.keras.layers.Input(name=colname, shape=(), dtype='string') 
#           for colname in sparse.keys()
# })

In [49]:
# NBUCKETS = 100
# DNN_HIDDEN_UNITS = [16, 8]

In [47]:
# ## 特征处理
# latbuckets = np.linspace(20.0, 50.0, NBUCKETS).tolist()  # USA
# lonbuckets = np.linspace(-120.0, -70.0, NBUCKETS).tolist() # USA
# disc = {}
# disc.update({
#        'd_{}'.format(key) : tf.feature_column.bucketized_column(real[key], latbuckets) 
#           for key in ['dep_lat', 'arr_lat']
# })
# disc.update({
#        'd_{}'.format(key) : tf.feature_column.bucketized_column(real[key], lonbuckets) 
#           for key in ['dep_lon', 'arr_lon']
# })

# # cross columns that make sense in combination
# sparse['dep_loc'] = tf.feature_column.crossed_column([disc['d_dep_lat'], disc['d_dep_lon']], NBUCKETS*NBUCKETS)
# sparse['arr_loc'] = tf.feature_column.crossed_column([disc['d_arr_lat'], disc['d_arr_lon']], NBUCKETS*NBUCKETS)
# sparse['dep_arr'] = tf.feature_column.crossed_column([sparse['dep_loc'], sparse['arr_loc']], NBUCKETS ** 4)
# #sparse['ori_dest'] = tf.feature_column.crossed_column(['origin', 'dest'], hash_bucket_size=1000)

# # embed all the sparse columns
# embed = {
#        'embed_{}'.format(colname) : tf.feature_column.embedding_column(col, 10)
#           for colname, col in sparse.items()
# }
# real.update(embed)

# # one-hot encode the sparse columns
# sparse = {
#     colname : tf.feature_column.indicator_column(col)
#           for colname, col in sparse.items()
# }

In [50]:
# # Build a wide-and-deep model.
# def wide_and_deep_classifier(inputs, linear_feature_columns, dnn_feature_columns, dnn_hidden_units):
#     deep = tf.keras.layers.DenseFeatures(dnn_feature_columns, name='deep_inputs')(inputs)
#     layers = [int(x) for x in dnn_hidden_units.split(',')]
#     for layerno, numnodes in enumerate(layers):
#         deep = tf.keras.layers.Dense(numnodes, activation='relu', name='dnn_{}'.format(layerno+1))(deep)        
#     wide = tf.keras.layers.DenseFeatures(linear_feature_columns, name='wide_inputs')(inputs)
#     both = tf.keras.layers.concatenate([deep, wide], name='both')
#     output = tf.keras.layers.Dense(1, activation='sigmoid', name='pred')(both)
#     model = tf.keras.Model(inputs, output)
#     model.compile(optimizer='adam',
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model
    
# model = wide_and_deep_classifier(
#     inputs,
#     linear_feature_columns = sparse.values(),
#     dnn_feature_columns = real.values(),
#     dnn_hidden_units = DNN_HIDDEN_UNITS)

# tf.keras.utils.plot_model(model, 'flights_model.png', show_shapes=False, rankdir='LR')

ResourceExhaustedError: OOM when allocating tensor with shape[100000000,10] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Add] name: deep_inputs/arr_lat_bucketized_X_arr_lon_bucketized_X_dep_lat_bucketized_X_dep_lon_bucketized_embedding/embedding_weights/Initializer/truncated_normal/